In [ ]:
#!/usr/bin/env python3
import tkinter as tk
from tkinter import BOTTOM
from datetime import datetime, date
import calendar
import webbrowser
from PIL import Image, ImageTk

UPDATE_INTERVAL = 1000
BOTTOM_IMAGE_PATH = "88/Picture/GS.png"  # <-- Make sure this path exists
DEFAULT_WEB_APPS = [
    ("Google Docs", "https://docs.google.com/"),
    ("ChatGPT", "https://chat.openai.com/"),
    ("YouTube", "https://www.youtube.com/"),
]

class NeonLauncher(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Neon Launcher")
        self.geometry("1000x740")
        self.resizable(True, True)

        # Theme
        self.fg = "#00FF77"
        self.bg = "#000000"
        self.highlight = "#FA00C4"
        self.label_font_family = "Courier"
        self.label_font_size = 20

        self.web_apps = DEFAULT_WEB_APPS.copy()
        self.bottom_image_refs = []

        self.today = date.today()
        self._build_ui()
        self.update_every_second()

        # Bind resize event to adjust bottom image
        self.bind("<Configure>", self._resize_bottom_image)

    def _build_ui(self):
        self.configure(bg=self.bg)

        # Top: clock & date
        top = tk.Frame(self, bg=self.bg)
        top.pack(fill="x", pady=8)
        self.clock_var = tk.StringVar()
        self.date_var = tk.StringVar()
        tk.Label(top, textvariable=self.clock_var, fg=self.fg, bg=self.bg,
                 font=(self.label_font_family, 30, "bold")).pack(side="left", padx=18)
        tk.Label(top, textvariable=self.date_var, fg=self.fg, bg=self.bg,
                 font=(self.label_font_family, 14)).pack(side="left", padx=6)

        # Main: Calendar and Web Apps
        main = tk.Frame(self, bg=self.bg)
        main.pack(fill="both", expand=True, padx=16, pady=20)

        # Left: Calendar
        self.cal_frame = tk.Frame(main, bg=self.bg)
        self.cal_frame.pack(side="left", fill="both", expand=True, padx=20, pady=20)
        self.cal_labels = []
        self.build_calendar()
        self.draw_calendar()

        # Right: Web Apps
        control = tk.Frame(main, bg=self.bg)
        control.pack(side="right", fill="both", expand=True, padx=20, pady=20)
        tk.Label(control, text="Web Apps", fg=self.fg, bg=self.bg,
                 font=(self.label_font_family, 18, "bold")).pack(anchor="w", pady=(4,6))
        self.web_grid = tk.Frame(control, bg=self.bg)
        self.web_grid.pack(fill="both", expand=True)
        self.build_grid(self.web_grid, self.web_apps, columns=2)

        # Bottom: image
        self.bottom_frame = tk.Frame(self, bg=self.bg, height=150)
        self.bottom_frame.pack(side=BOTTOM, fill="x")
        try:
            self.original_bottom_image = Image.open(BOTTOM_IMAGE_PATH)
            self.bottom_label = tk.Label(self.bottom_frame, bg=self.bg)
            self.bottom_label.pack(fill="both", expand=True)
            self._resize_bottom_image()
        except Exception as e:
            print("Error loading bottom image:", e)

    # Resize bottom image dynamically
    def _resize_bottom_image(self, event=None):
        if hasattr(self, "original_bottom_image"):
            width = self.bottom_frame.winfo_width() or 1000
            aspect_ratio = self.original_bottom_image.height / self.original_bottom_image.width
            resized_image = self.original_bottom_image.resize((width, int(width * aspect_ratio)), Image.LANCZOS)
            tk_image = ImageTk.PhotoImage(resized_image)
            self.bottom_image_refs.append(tk_image)
            self.bottom_label.config(image=tk_image)
            self.bottom_label.image = tk_image  # Keep reference

    # Calendar
    def build_calendar(self):
        days = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
        for i, d in enumerate(days):
            tk.Label(self.cal_frame, text=d, fg=self.fg, bg=self.bg,
                     font=(self.label_font_family, 16, "bold")).grid(row=0, column=i)
        for r in range(1,7):
            row = []
            for c in range(7):
                lbl = tk.Label(self.cal_frame, text="", width=4, height=2,
                               fg=self.fg, bg=self.bg,
                               font=(self.label_font_family, 20))
                lbl.grid(row=r, column=c)
                row.append(lbl)
            self.cal_labels.append(row)

    def draw_calendar(self):
        now = datetime.now()
        month_days = calendar.Calendar().monthdayscalendar(now.year, now.month)
        for r, week in enumerate(month_days):
            for c, day in enumerate(week):
                lbl = self.cal_labels[r][c]
                lbl.config(text=str(day) if day != 0 else "")
                lbl.config(bg=self.highlight if day == self.today.day else self.bg)

    # Web Apps
    def build_grid(self, frame, items, columns=2):
        for w in frame.winfo_children():
            w.destroy()
        for idx, (label, url) in enumerate(items):
            r, c = divmod(idx, columns)
            btn = tk.Button(frame, text=label, fg=self.fg, bg=self.bg,
                            font=(self.label_font_family, self.label_font_size),
                            relief="flat", command=lambda u=url: webbrowser.open(u))
            btn.grid(row=r, column=c, padx=6, pady=6, sticky="nsew")
            btn.bind("<Enter>", lambda e, b=btn: b.config(bg="#111"))
            btn.bind("<Leave>", lambda e, b=btn: b.config(bg=self.bg))
        for col in range(columns):
            frame.grid_columnconfigure(col, weight=1)

    # Clock & calendar update
    def update_every_second(self):
        now = datetime.now()
        self.clock_var.set(now.strftime("%I:%M:%S %p").lstrip("0"))
        self.date_var.set(now.strftime("%A, %B %d, %Y"))
        self.draw_calendar()
        self.after(UPDATE_INTERVAL, self.update_every_second)

if __name__ == "__main__":
    app = NeonLauncher()
    app.mainloop()
